In [6]:
deployment = "gpt4"
model = "gpt-4"

# 回顾

In [7]:
from langchain.chains import LLMRequestsChain
from langchain import PromptTemplate, OpenAI, LLMChain
from langchain.chat_models import AzureChatOpenAI
llm = AzureChatOpenAI(deployment_name = deployment, model_name=model, temperature=0, max_tokens=200)
template = """Between >>> and <<< are the raw search result text from web.
Extract the answer to the question '{query}' or say "not found" if the information is not contained.
Use the format
Extracted:<answer or "not found">
>>> {requests_result} <<<
Extracted:"""

PROMPT = PromptTemplate(
  input_variables=["query", "requests_result"],
  template=template,
)


query_chain = LLMRequestsChain(llm_chain = LLMChain(llm=llm, prompt=PROMPT), output_key="query_info")


In [8]:
from langchain import PromptTemplate, OpenAI, LLMChain
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import SequentialChain

translating_prompt_template = """
translate "{query_info}" into English:

"""

translating_chain = LLMChain(
    llm = llm,
    prompt=PromptTemplate.from_template(translating_prompt_template),
    output_key = "translated"
)




In [6]:
llm = AzureChatOpenAI(deployment_name = deployment, model_name=model, temperature=0, max_tokens=200)

def overall(question):
    inputs = {
      "query": question,
      "url": "http://www.baidu.com/s?wd=" + question.replace(" ", "+")
      # "url": "https://cn.bing.com/search?q=" + question.replace(" ", "+")
    }
    
    overall_chain = SequentialChain(
        chains=[query_chain, translating_chain],
        input_variables=["query","url"],
        output_variables=["translated"],
        verbose=True
    )
    
    return overall_chain(inputs)["translated"]

overall("北京今天天气")



> Entering new SequentialChain chain...

> Finished chain.


'"20°C with moderate rain, east wind at level 4, temperature ranging from 20~23°C"'

# 没有记忆的模型

In [7]:
import openai
def get_response(input):
    response = openai.ChatCompletion.create(
        engine=deployment, # engine = "deployment_name".
        messages=[
            {"role": "system", "content": "You are an AI assistant."},   
            {"role": "user", "content": input}
        ],
        temperature = 0.9, 
        max_tokens = 200
      )
    return response.choices[0].message.content

In [8]:
print(get_response("你好，今天是周二我要去健身，我一般每周二健身。你今天干什么？"))
print(get_response("我一般周几健身？"))

作为一个AI助手，我并不会有特定的日程安排。我24小时全天候在线，随时都可以为你提供服务。无论你需要记事、设定闹钟、查找信息或其他任何帮助，我都将尽力为你解决。祝你今天的健身训练顺利！
对不起，我无法回答这个问题。作为一个AI，我没有实时访问或跟踪您的个人日程或习惯。我的设计是为了保护您的隐私和数据安全。


# 通过Gradio快速展示功能

In [9]:
!pip install gradio

In [10]:
import gradio as gr
def respond(message, chat_history):
        bot_message = get_response(message)
        chat_history.append((message, bot_message)) #保存历史对话记录，用于显示
        return "", chat_history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=240) #对话框
    msg = gr.Textbox(label="Prompt") #输入框
    btn = gr.Button("Submit") #提交按钮
    #提交
    btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot]) 
gr.close_all()
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


# 提供外部记忆

In [12]:
import openai
def get_response(msg):
    # print(msg)
    response = openai.ChatCompletion.create(
        engine=deployment, # engine = "deployment_name".
        messages=msg,
        temperature = 0.9, 
        max_tokens = 600
    )
    return response.choices[0].message.content

In [13]:

def history_to_prompt(chat_history): # 将对话内容保存在一个List里
    msg = [{"role": "system", "content": "You are an AI assistant."}]
    i = 0
    for round_trip in chat_history: # 将List里的内容，组成 ChatCompletion的 messages部分，{role，content} dict
        msg.append({"role": "user", "content": round_trip[0]})
        msg.append({"role": "assistant", "content": round_trip[1]})
    return msg

def respond(message, chat_history):
    his_msg = history_to_prompt(chat_history) #并装历史会话，ChatCompletion的 messages部分格式
    his_msg.append({"role": "user", "content": message}) # 放入当前用户问题
    bot_message = get_response(his_msg)
    chat_history.append((message, bot_message)) # 将用户问题和返回保存到 历史记录 List
    return "", chat_history



In [15]:
import gradio as gr
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=480) #just to fit the notebook
    msg = gr.Textbox(label="Prompt")
    btn = gr.Button("Submit")
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

    btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot]) #Press enter to submit
gr.close_all()
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7863
Running on public URL: https://53f21db72a477bb701.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
